In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

## GroupBy, Aggregation, Apply

Grouping data together and doing aggregation or applying other functions to data.  The way it works is a sort of map-reduce method, descibred as "split", "apply", and "combine".   So "split" and "apply" are both the "map" stage, and "combine" is the "reduce" stage.

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)
                  })
df

,key1,key2,data1,data2
0,a,one,-0.571247,-0.121405
1,a,two,-0.506409,1.134927
2,b,one,-0.056945,0.830911
3,b,two,-0.965405,-0.135428
4,a,one,-0.369866,-0.118589


In [3]:
df['data1'].groupby(df['key1']).mean()

key1
a   -0.482507
b   -0.511175
Name: data1, dtype: float64

In [4]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.470557
      two    -0.506409
b     one    -0.056945
      two    -0.965405
Name: data1, dtype: float64

In [5]:
means.unstack()

key2,one,two
key1,,
a,-0.470557,-0.506409
b,-0.056945,-0.965405


In addition to grouping a column (Series), we can also group a whole data frame by one of its columns.

In [6]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.482507,0.298311
b,-0.511175,0.347742


Note above that there is no "key2" column.  Because "key2" is non-numeric it is a nuisance column for a numerical aggregation operation such as `mean`.

In [7]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.470557 -0.119997
     two  -0.506409  1.134927
b    one  -0.056945  0.830911
     two  -0.965405 -0.135428

It is generally useful to get the `size()` of your groups, maybe to see what your data looks like in grouped form.

In [8]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [9]:
grouping = df.groupby(['key1', 'key2'])
grouping

In [10]:
pd.Series([attr_name for attr_name in dir(grouping) if attr_name[0] != '_'])

0           agg
1     aggregate
2           all
3           any
4         apply
        ...    
65         tail
66         take
67    transform
68       tshift
69          var
Length: 70, dtype: object

### Iterating groups
A groupby object supports iteration.

In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.571247 -0.121405
1    a  two -0.506409  1.134927
4    a  one -0.369866 -0.118589
b
  key1 key2     data1     data2
2    b  one -0.056945  0.830911
3    b  two -0.965405 -0.135428


When grouping by multiple columns, the first returned item in the iteration will be a tuple, as illustrated below.

In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))

('a', 'one')
('a', 'two')
('b', 'one')
('b', 'two')


The ability to iterate over groups allows you to divide the data in convenient ways.

In [13]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.056945,0.830911
3,b,two,-0.965405,-0.135428


You can group by other axis, and you only need a list or series of the same length.

In [14]:
for name, group in df.groupby(df.dtypes, axis=1):
    print(name)
    print(group)

float64
      data1     data2
0 -0.571247 -0.121405
1 -0.506409  1.134927
2 -0.056945  0.830911
3 -0.965405 -0.135428
4 -0.369866 -0.118589
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


A DataFrameGroupBy object can be indexed by a column name, meaning that the following two produce the same result:

In [15]:
df.groupby('key1')['data1'].mean()

key1
a   -0.482507
b   -0.511175
Name: data1, dtype: float64

In [16]:
df['data1'].groupby(df['key1']).mean()

key1
a   -0.482507
b   -0.511175
Name: data1, dtype: float64

For large datasets, it may be desirable to only aggregate selected columns.

In [17]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.119997
     two   1.134927
b    one   0.830911
     two  -0.135428

Note that the object returned from this grouping is a DataFrameGroupBy if a list is passed when indexing and a SeriesGroupBy if a scalar value is passeed when indexing.

In [18]:
df.groupby('key1')['data1']

In [19]:
df.groupby('key1')[['data1']]

### Grouping with Dicts and Series

Using another example dataframe to illustrate grouping by something other than an array (np or pd).

In [20]:
people = pd.DataFrame(np.random.randn(5, 5), columns=list('abcde'), index='Joe,Steve,Wes,Jim,Travis'.split(','))
people.iloc[2:3, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,-1.214199,-0.561205,0.607995,0.605433,1.247509
Steve,-1.399416,1.401355,-0.776024,-0.068348,0.885591
Wes,0.598584,NaN,NaN,-0.965313,0.899901
Jim,1.308463,0.388420,1.353430,0.717802,0.074013
Travis,0.983351,1.074930,1.565522,-0.469288,0.373819


We have a dict that categories the columns by some other criteria. We could have added this (hierarchical indexing) to the dataframe, but we don't have to - we can have it outside the dataframe.  Note that the mapping from `'f'` is not used.

In [21]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,1.213428,-0.527895
Steve,-0.844371,0.887530
Wes,-0.965313,1.498485
Jim,2.071232,1.770896
Travis,1.096235,2.432100


This can also be done by using a `pd.Series` as a mapping

In [22]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [23]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

Using a function is a more general mechanism than using a mapping. The data passed to the fuction is the name of the index or columns, and the function's return value is used as the group key.  For example, maybe we have a long list of names and a count of population with those names.  We want to find out how many people have names of length 3 characters, for instance.

In [24]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.692848,-0.172785,1.961425,0.357922,2.221422
5,-1.399416,1.401355,-0.776024,-0.068348,0.885591
6,0.983351,1.074930,1.565522,-0.469288,0.373819


Based on our example, the data for "Joe", "Wes", and "Jim" all are mapped to the group `3`.

We can mix mapping by arrays, lists, dicts, and functions because everything is internally converted into arrays.  This utilizes hierarchical indexing.

In [25]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.214199 -0.561205  0.607995 -0.965313  0.899901
  two  1.308463  0.388420  1.353430  0.717802  0.074013
5 one -1.399416  1.401355 -0.776024 -0.068348  0.885591
6 two  0.983351  1.074930  1.565522 -0.469288  0.373819

### Grouping by Index Levels

If we have a dataframe with hierarchical indexing, we can group by any level (or levels) of the hierarchical index.

In [26]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      2.494697 -0.720841 -0.841977 -0.855289 -0.136624
1     -0.526472 -2.056042 -1.103926 -0.389779 -0.254939
2     -0.013055 -1.266205  0.027709 -1.223359  1.700055
3      1.256652 -0.718723  0.650422 -0.386283  1.115268

To group by a level, pass the name of the level to groupby.

In [27]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


Or the number of the level:

In [28]:
hier_df.groupby(level=0, axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

We've seen examples of a number of aggregations, such as `mean`, `sum`, and `count`. We can use more of these or even define our own.

In [29]:
df

,key1,key2,data1,data2
0,a,one,-0.571247,-0.121405
1,a,two,-0.506409,1.134927
2,b,one,-0.056945,0.830911
3,b,two,-0.965405,-0.135428
4,a,one,-0.369866,-0.118589


In [30]:
df.groupby('key1').quantile()

,data1,data2
key1,,
a,-0.506409,-0.118589
b,-0.511175,0.347742


To use your own aggregation method, pass any function that operates on an array of values to the `aggregate` or `agg` method.

In [31]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

df.groupby('key1').agg(peak_to_peak)

,data1,data2
key1,,
a,0.201381,1.256333
b,0.908460,0.966339


Some methods work that are not even aggregations, strictly speaking.  The below capability can by very powerful for analysis.

In [32]:
df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.482507  0.102796 -0.571247 -0.538828 -0.506409 -0.438137   
b      2.0 -0.511175  0.642378 -0.965405 -0.738290 -0.511175 -0.284060   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.369866   3.0  0.298311  0.724532 -0.121405 -0.119997 -0.118589   
b    -0.056945   2.0  0.347742  0.683305 -0.135428  0.106157  0.347742   

                          
           75%       max  
key1                      
a     0.508169  1.134927  
b     0.589327  0.830911

Non-optimized aggregations can be **much** slower than optimized operations, because an intermediate array for each group needs to be constructed.

### Column wise and Multiple Function Application

In [33]:
tips = pd.read_csv('pydata-book/examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


You may want to group by a different function depending on the column, or multiple functions at once. This is illustrated below.

In [34]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

The ability to calculate the pre-defined mean by passing a string also allows you to pass more than one aggregation for this column.

In [35]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

You can rename the columns by passing a list of tuples.

In [36]:
grouped_pct.agg(['mean', 'std', ('spread', peak_to_peak)])

mean       std    spread
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

Now that this is clear on a SeriesGroupBy, we can apply the same kind of thing to a DataFrame.  Suppose we want to apply the same functions to the `tip_pct` and `total_bill` columns.

In [37]:
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

As you might have guessed by now, passing a `dict` to the `agg` method will do what you think, and allow you to apply a different aggregation to different columns.

In [38]:
grouped.agg({'tip': 'max', 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

And you can compute multiple aggregations as well.

In [39]:
grouped.agg({'tip_pct': [('average', 'mean'), 'max', 'min', 'std'], 'size': 'sum'})

tip_pct                               size
              average       max       min       std  sum
day  smoker                                             
Fri  No      0.151650  0.187735  0.120385  0.028123    9
     Yes     0.174783  0.263480  0.103555  0.051293   31
Sat  No      0.158048  0.291990  0.056797  0.039767  115
     Yes     0.147906  0.325733  0.035638  0.061375  104
Sun  No      0.160113  0.252672  0.059447  0.042347  167
     Yes     0.187250  0.710345  0.065660  0.154134   49
Thur No      0.160298  0.266312  0.072961  0.038774  112
     Yes     0.163863  0.241255  0.090014  0.039389   40

A DataFrame will have hierarchical columns only if multiple aggregation functions are applied to at least one column.

### Returning Aggregated Data without row indexes

Suppose you do not want the group by keys to be part of the index of the resulting `Series` or `DataFrame`, but to remain as columns of a DataFrame.  Simply pass `as_index=False` when grouping.

In [40]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## General split-apply-combine

Group by is basically doing the "split" part of the operation, and when you call `apply`, that's what you are doing.  Then, pandas uses `pd.concat` to stitch the pieces together.  So, all you really need is a function that works properly on pandas objects.

In [41]:
def top(df, column='tip_pct', n=4):
    return df.sort_values(by=column)[-n:]

Since this function accepts a `DataFrame` it is appropriate for apply in DataFrameGroupBy result.

In [42]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

If you want to pass other arguments to the function, supply them after the function in the call to apply (guessing these should be kwarg for readability)

In [43]:
tips.groupby(['smoker', 'day']).apply(top, n=1)

total_bill   tip smoker   day    time  size   tip_pct
smoker day                                                            
No     Fri  223       15.98  3.00     No   Fri   Lunch     3  0.187735
       Sat  232       11.61  3.39     No   Sat  Dinner     2  0.291990
       Sun  51        10.29  2.60     No   Sun  Dinner     2  0.252672
       Thur 149        7.51  2.00     No  Thur   Lunch     2  0.266312
Yes    Fri  93        16.32  4.30    Yes   Fri  Dinner     2  0.263480
       Sat  67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       Sun  172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       Thur 194       16.58  4.00    Yes  Thur   Lunch     2  0.241255

If you don't want the category to be moved into the DataFrame index, that's OK.  Just pass group_keys=False and it will not be.

In [44]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

`groupby` goes well with `pd.cut` and `pd.qcut` to do quantile and bucket analysis.

In [45]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})
frame

,data1,data2
0,0.619196,0.018154
1,0.227273,-1.991437
2,0.170825,0.331069
3,0.777744,1.114885
4,-0.158957,-0.489244
...,...,...
995,0.278568,0.198460
996,0.692573,-0.735794
997,0.652590,-0.952392
998,-0.266207,-0.023577


`pd.cut(series, 4)` will (a) determine quartile categories, and (b) map to a series that puts each record into one of those four categories.  The categories are a special sort of `Category` object.

In [46]:
pd.cut(frame.data1, 4)

0      (-0.182, 1.187]
1      (-0.182, 1.187]
2      (-0.182, 1.187]
3      (-0.182, 1.187]
4      (-0.182, 1.187]
            ...       
995    (-0.182, 1.187]
996    (-0.182, 1.187]
997    (-0.182, 1.187]
998    (-1.55, -0.182]
999    (-0.182, 1.187]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64]): [(-2.924, -1.55] < (-1.55, -0.182] < (-0.182, 1.187] < (1.187, 2.555]]

Now that we have these categories for the data, we can group by them and do analysis within each quantile.

In [47]:
quartiles = pd.cut(frame.data1, 4)
frame.data2.groupby(quartiles).mean()

data1
(-2.924, -1.55]    0.096557
(-1.55, -0.182]   -0.042695
(-0.182, 1.187]    0.011133
(1.187, 2.555]     0.148349
Name: data2, dtype: float64

In [48]:
def get_stats(grouped):
    return {'min': grouped.min(), 'max': grouped.max(), 'count': grouped.count(), 'max': grouped.max()}

In [49]:
frame.data2.groupby(quartiles).apply(get_stats).unstack()

,min,max,count
data1,,,
"(-2.924, -1.55]",-2.480682,2.952775,53.0
"(-1.55, -0.182]",-2.886848,2.722420,366.0
"(-0.182, 1.187]",-2.968463,2.885578,457.0
"(1.187, 2.555]",-2.409153,2.856506,124.0


This is how quantiles are usually used - produce categorical data from one random variable, `data1`, that is expected to have some relationship to another random variable, `data2`.  This is in the case where the distribution isn't quite close enough to normal, I guess.

These were equal length buckets, to get equal size buckets, use `pd.qcut`.

In [50]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouping

0      7
1      5
2      5
3      7
4      4
      ..
995    6
996    7
997    7
998    3
999    8
Name: data1, Length: 1000, dtype: int64

In [51]:
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count
data1,,,
0,-2.886848,2.952775,100.0
1,-2.829965,2.390807,100.0
2,-2.818223,2.292053,100.0
3,-2.003523,2.722420,100.0
4,-2.968463,2.683028,100.0
5,-2.622810,1.941403,100.0
6,-2.145228,2.477108,100.0
7,-2.058812,2.885578,100.0
8,-2.409153,1.912146,100.0


### Example: Filling Missing Values with Group-specific Values

Filling missing values can get fancy. Consider the following:

In [52]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.532998
2         NaN
3   -0.192130
4         NaN
5   -0.139897
dtype: float64

In [53]:
s.fillna(s.mean())

0   -0.288341
1   -0.532998
2   -0.288341
3   -0.192130
4   -0.288341
5   -0.139897
dtype: float64

Suppose you need the fill value to vary by group?  One way to do this is to group the data and then call `apply` with a function that computes the fill value and then calls `fillna` on the group.

In [54]:
states = 'Ohio,New York,Vermont,Florida,Oregon,Nevada,California,Idaho'.split(',')
group_key = ['East']*4 + ['West']*4

In [55]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.131952
New York     -0.048197
Vermont       0.589556
Florida      -0.100202
Oregon        1.030325
Nevada       -0.578393
California    0.116579
Idaho        -0.146144
dtype: float64

Let's make some values `NaN`

In [56]:
data[['Vermont','Nevada','Ohio']] = np.nan
data

Ohio               NaN
New York     -0.048197
Vermont            NaN
Florida      -0.100202
Oregon        1.030325
Nevada             NaN
California    0.116579
Idaho        -0.146144
dtype: float64

In [57]:
data.groupby(group_key).mean()

East   -0.074199
West    0.333586
dtype: float64

In [58]:
fillmean = lambda g: g.fillna(g.mean())

In [59]:
data.groupby(group_key).apply(fillmean)

Ohio         -0.074199
New York     -0.048197
Vermont      -0.074199
Florida      -0.100202
Oregon        1.030325
Nevada        0.333586
California    0.116579
Idaho        -0.146144
dtype: float64

You might also have predefined values to be used for each group. These can easily be used.

In [60]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.500000
New York     -0.048197
Vermont       0.500000
Florida      -0.100202
Oregon        1.030325
Nevada       -1.000000
California    0.116579
Idaho        -0.146144
dtype: float64

### Example: Random sampling and permutation

You can use a random value to group data, to do random sampling, permutation, or divide data into a test set and a sample set.  The example is for playing cards.

In [61]:
suits = ['H', 'S', 'C', 'D'] # hearts, spades, clubs, diamonds
card_vals = (list(range(1, 11)) + [10]*3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'Q', 'K']
card_names = []
for suit in suits:
    card_names += [str(num) + suit for num in base_names]
deck = pd.Series(card_vals, index=card_names)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

Drawing a hand of five cards could be as simple as calling the sample method with n=5.

In [62]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck, 5)

5S      5
KS     10
10C    10
AC      1
7C      7
dtype: int64

Suppose we have a special game where we need to draw 2 cards from each suit.  Here, we can use `groupby` to group by the suit (which is the last character of the index name).

In [63]:
deck.groupby(lambda card: card[-1]).apply(draw, n=2)

C  QC     10
   10C    10
D  7D      7
   KD     10
H  QH     10
   5H      5
S  6S      6
   QS     10
dtype: int64

Alternatively, we could omit the group keys since the group is already part of the index column.

In [64]:
deck.groupby(lambda card: card[-1], group_keys=False).apply(draw, 2)

5C    5
2C    2
6D    6
8D    8
9H    9
2H    2
6S    6
4S    4
dtype: int64

See `deck.sample?` for more information.

### Example: Group Weighted Average and Correlation

Since "split-apply-combine" operates on an entire row (or column) of a `DataFrame`, it is possible to use data from multiple columns in an operation.  Consider this example where the DataFrame contains a group key, value, and weight.

In [65]:
df = pd.DataFrame({'category': list('aaaabbbb'), 'data': 3+np.random.randn(8), 'weights': np.random.rand(8)})
df

,category,data,weights
0,a,2.842560,0.183946
1,a,1.629880,0.227909
2,a,1.733584,0.702620
3,a,2.082830,0.965250
4,b,1.819608,0.198462
5,b,2.565713,0.644230
6,b,4.498174,0.559648
7,b,2.576267,0.139621


Group weighted average by category would then be.

In [66]:
grouped = df.groupby('category')
group_wavg = lambda g: np.average(g.data, weights=g.weights)
grouped.apply(group_wavg)

category
a    1.982399
b    3.172017
dtype: float64

As another example, consider some financial data from Yahoo containing end-of-day prices for a few stocks and the S&P 500 index.

In [67]:
close_px = pd.read_csv('pydata-book/examples/stock_px_2.csv', parse_dates=True, index_col=0)
close_px

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
...,...,...,...,...
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66


One task of interest would be to compute how well the performance of the individual stocks correlate with the S&P 500.  We do that by computing the yearly correlation of daily returns with SPX.  We will do this by using:
* `DataFrame.pct_change` to compute the percent change in each column.
* `Series.corrwith` to compute the correlation of one column with another.

In [68]:
returns = close_px.pct_change().dropna()
returns

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


In [69]:
spx_corrwith = lambda x: x.corrwith(x['SPX'])
get_year = lambda d: d.year
by_year = returns.groupby(get_year)
by_year.apply(spx_corrwith)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


We can also easily compute the correlation between two stocks

In [70]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

Note how accessing a column of the grouped dataframe reduces the dimension and we are left with a `Series`.

### Group-wise Linear Regression

As the previous examples suggest, the function you pass to apply works as long as it returns a scalar or a pandas object, e.g. `Series` or `DataFrame`.  So, we can do more complicated stastical analysis and in this example Wes McKinney does an OLS (ordinary least squares) linear regression on some data within each group.

From a high-level perspective, this assumes that there is a linear relationship between the symbol AAPL and SPX, and SPX is treated as the independent variable and AAPL is the random variable (statistics terminology) we want to predict. The goal is to fit a straight line that is a function of SPX to the AAPL gains while minimizing error through the OLS method.

Before we uses statsmodels group-wise to do an OLS regression, let us try to do it to the whole data.

The random variable we want to predict is a simple Series.

In [71]:
Y = returns['AAPL']
Y

2003-01-03    0.006757
2003-01-06    0.000000
2003-01-07   -0.002685
2003-01-08   -0.020188
2003-01-09    0.008242
                ...   
2011-10-10    0.051406
2011-10-11    0.029526
2011-10-12    0.004747
2011-10-13    0.015515
2011-10-14    0.033225
Name: AAPL, Length: 2213, dtype: float64

The independent variable has to be a DataFrame because we need to add an intercept column.  We have to use `.loc` because otherwise it would be a slice of the data and that is not good as we are adding a column.

In [72]:
X = returns.loc[:, ['SPX']]
X['intercept'] = 1.0
X

,SPX,intercept
2003-01-03,-0.000484,1.0
2003-01-06,0.022474,1.0
2003-01-07,-0.006545,1.0
2003-01-08,-0.014086,1.0
2003-01-09,0.019386,1.0
...,...,...
2011-10-10,0.034125,1.0
2011-10-11,0.000544,1.0
2011-10-12,0.009795,1.0
2011-10-13,-0.002974,1.0


Now we can do the regression and print a summary.

In [73]:
result = sm.OLS(Y, X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   AAPL   R-squared:                       0.319
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     1034.
Date:                Thu, 22 Jul 2021   Prob (F-statistic):          1.91e-186
Time:                        14:23:26   Log-Likelihood:                 5494.4
No. Observations:                2213   AIC:                        -1.098e+04
Df Residuals:                    2211   BIC:                        -1.097e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SPX            1.0260      0.032     32.155      0.000       0.963       1.089
intercept      0.0019      0.000      4.412      0.000       0.001       0.003
==============================================================================
Omnibus:                      451.599   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3369.292
Skew:                           0.756   Prob(JB):                         0.00
Kurtosis:                       8.852   Cond. No.                         74.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [74]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.0
    result = sm.OLS(Y, X).fit()
    return result.params
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


### Pivot Tables and Cross Tabulation

Pivot tables with pandas are made possible using the groupby functionality described in this chapter and reshape operations with hierarchical indexing. Each `DataFrame` has a `pivot_table` method and there is also a function `pd.pivot_table`.

In [75]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


We can pivot the values of the categorical columns "day" and "smoker" out, and use the default behavior of selecting group means on the data. The "time" column goes away since it is non-numeric data.

In [76]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

Suppose we only want to compute the mean for "tip_pct" and "size" columms, and we also want to preserve "smoker" as a hierarchical value in the columns.

In [77]:
tips.pivot_table(['tip_pct', 'size'], index=['day', 'time'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
day  time                                          
Fri  Dinner  2.000000  2.222222  0.139622  0.165347
     Lunch   3.000000  1.833333  0.187735  0.188937
Sat  Dinner  2.555556  2.476190  0.158048  0.147906
Sun  Dinner  2.929825  2.578947  0.160113  0.187250
Thur Dinner  2.000000       NaN  0.159744       NaN
     Lunch   2.500000  2.352941  0.160311  0.163863

By adding `margins=True` we cause pandas to compute the mean also for non-categorized data add an "All" row and "All" columns both for "size" and "tip_pct".

In [78]:
tips.pivot_table(['tip_pct', 'size'], index=['day', 'time'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
day  time                                                              
Fri  Dinner  2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
     Lunch   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
Sat  Dinner  2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
Sun  Dinner  2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
Thur Dinner  2.000000       NaN  2.000000  0.159744       NaN  0.159744
     Lunch   2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

We can compute a different aggregation by passing it as the `aggfunc` argument.  We can pass a name like "count" or a function such as `len`.

In [79]:
tips.pivot_table(['tip_pct', 'size'], index=['day', 'time'], columns='smoker', aggfunc='count')

size       tip_pct      
smoker         No   Yes      No   Yes
day  time                            
Fri  Dinner   3.0   9.0     3.0   9.0
     Lunch    1.0   6.0     1.0   6.0
Sat  Dinner  45.0  42.0    45.0  42.0
Sun  Dinner  57.0  19.0    57.0  19.0
Thur Dinner   1.0   NaN     1.0   NaN
     Lunch   44.0  17.0    44.0  17.0

In [80]:
tips.pivot_table(['tip_pct', 'size'], index=['day', 'time'], columns='smoker', aggfunc=len)

size       tip_pct      
smoker         No   Yes      No   Yes
day  time                            
Fri  Dinner   3.0   9.0     3.0   9.0
     Lunch    1.0   6.0     1.0   6.0
Sat  Dinner  45.0  42.0    45.0  42.0
Sun  Dinner  57.0  19.0    57.0  19.0
Thur Dinner   1.0   NaN     1.0   NaN
     Lunch   44.0  17.0    44.0  17.0

### Cross Tabulations

A `crosstab` is a special sort of pivot table that computes group frequencies (e.g. counts). So, it is a lot like what we did above with the "count" aggfunc, but with a more convenient interface. The arguments for crosstab can be arrays (np.array), Series, or lists of arrays/Series.  Just like any pivot_table, you can compute margins.

In [81]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

## Advanced GroupBy Use

Chapter 12 (Advanced Pandas) covers some scenarios of interest.

### Group transforms and "unwrapped" groupby

There is a nother function `transform`, similar to `apply` but stricter about what you can do.  It replaces the values in the group, if a scalar value is returned it is broadcast to all grouped rows, and if a pandas object/array is returned it must be of the same shape as the group (so an array, list, or Series I guess).

This can be used to optimize some operations to do array arithmetic.

In [82]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4, 'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [83]:
df.groupby('key').transform(lambda g: g.mean())

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


We can also specify the name of a well known function.

In [84]:
df.groupby('key').transform('mean')

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


Or return an object of the same shape (without changing the grouped data)

In [86]:
df.groupby('key').transform(lambda g: g * 2)

,value
0,0.0
1,2.0
2,4.0
3,6.0
4,8.0
5,10.0
6,12.0
7,14.0
8,16.0
9,18.0


This transformation works within the group, so that group specifics can be done.

In [87]:
df.groupby('key').transform(lambda g: g.rank(ascending=False))

,value
0,4.0
1,4.0
2,4.0
3,3.0
4,3.0
5,3.0
6,2.0
7,2.0
8,2.0
9,1.0


We could also normalize within the group.

In [88]:
def normalize(x):
    return (x - x.mean())/x.std()

In [89]:
df.groupby('key').transform(normalize)

,value
0,-1.161895
1,-1.161895
2,-1.161895
3,-0.387298
4,-0.387298
5,-0.387298
6,0.387298
7,0.387298
8,0.387298
9,1.161895


This motivates the example of "unwrapping" the grouped math. If there are lots of smaller groups (arrays), the calculation steps can be done once on the whole array for a performance increase.

In [97]:
g = df.groupby('key')['value']
normalized = (df.value - g.transform('mean'))/g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64